<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="../assets/holoviz-logo-unstacked.svg" />
<div style="float:right; vertical-align:text-bottom;"><h2>Tutorial 3: Interlinked Panels</h2></div>

In [1]:
import panel as pn
from load_data import *

pn.extension()
df = load_data()

In [2]:
df.head()

,station,latitude,longitude,time,wvht,wspd,gst
0,41001,34.675,-72.698,2021-01-01T00:40:00Z,2.14,10.0,12.8
1,41001,34.675,-72.698,2021-01-01T01:40:00Z,2.23,10.6,12.9
2,41001,34.675,-72.698,2021-01-01T02:40:00Z,2.07,10.6,13.3
3,41001,34.675,-72.698,2021-01-01T03:40:00Z,1.97,9.2,11.6
4,41001,34.675,-72.698,2021-01-01T04:40:00Z,1.94,9.2,11.3


In the previous section we learned the very basics of working with Panel. Specifically we looked at the different types of components, how to update them and how to serve a Panel application or dashboard. However to start building actual apps with Panel we need to be able to add interactivity by linking different components together. In this section we will learn how to link widgets to outputs to start building some simple interactive applications.

In this section we will once again make use of the wave heights dataset we loaded previously and compute some statistics:

## Widgets and reactive components

`pn.interact` constructs widgets automatically that can then be reconfigured, but if you want more control, you'll want to instantiate widgets explicitly. A widget is an input control that allows a user to change a ``value`` using some graphical UI. A simple example is a `RangeSlider`:

In [ ]:
wvht_filter = pn.widgets.RangeSlider(name='Wave Heights', start=0, end=df['wvht'].max())

wvht_filter

Here the widget `value` is a [Parameter](https://param.pyviz.org) that is set to a tuple of the selected upper and lower bound. Parameters are an extended type of Python attribute that declare their type, range, etc. so that other code can interact with them in a consistent way. When we change the range using the widget the ``value`` parameter updates, and vice versa if you change the value parameter manually:

In [ ]:
wvht_filter.value

## Callbacks

The `depends` API is still a very high level way of declaring interactive components. Panel also supports the more low-level approach of writing callbacks in response to changes in some parameter, e.g. the ``value`` of a widget. All parameters can be watched using the ``.param.watch`` API, which will call the provided callback with an event object containing the old and new value of the widget.

Now that it is loaded we will create a slider which we will eventually use to select the row of the dataframe that we want to display:

In [ ]:
row_slider = pn.widgets.IntSlider(value=0, start=0, end=len(df)-1)

Next we create a Pane to display the current row of the dataframe with times formatted nicely:

In [ ]:
row_pane = pn.panel(df.loc[row_slider.value])

Now that we have defined both the widget and the object we want to update we can declare a callback to link the two. As we learned in the previous section assigning a new value to the ``object`` of a pane will update the display. In the callback we select the row of the dataframe and then assign it to the ``pane.object``.

In [ ]:
def df_callback(event):
    row_pane.object = df.loc[event.new]

Lastly we actually have to register this callback. To do so we provide the callback and the parameter we want to trigger the event on the slider's ``.param.watch`` method:

In [ ]:
row_slider.param.watch(df_callback, 'value')

Now that everything is connected up we can put both the widget and the pane in a panel and display them:

In [ ]:
pn.Column(row_slider, row_pane, width=400)

As you can see, this process is slightly more laborious than `pn.interact` or even the `pn.depends` approach, but doing it in this way should help you see how everything fits together and can be useful to more precisely control callbacks that update particular parameters or the contents of a larger layout.

# Moving onwards

Now that we have learned to link parameters between displayed objects and build interactive components, we can start building actual apps and dashboards. Before we move on to plotting and visualization let us quickly use what we have learned by adding interactivity to [the dashboard we built in the previous exercise](./exercises/Building_a_Dashboard.ipynb).